In [1]:
import requests
import pandas as pd
import time
import gdown
import os
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_data = 'drive/My Drive/PIP 2025 /Données propres/TEMP Catastrophes naturelles/'
path_export = 'drive/My Drive/PIP 2025 /Données propres/'

In [4]:
# Charger le fichier CSV incendies
df_incendies = pd.read_csv(path_data+'catastrophes_nat_incendies.csv',dtype={"code_insee": str})

In [5]:
# Charger les fichiers CSVs georisques mensuels et annuels
df_georisques_mensuel = pd.read_csv(path_data+'catastrophes_nat_georisques_mensuel.csv',dtype={"code_insee": str})
df_georisques_annuel = pd.read_csv(path_data+'catastrophes_nat_georisques_annuel.csv',dtype={"code_insee": str})

<ipython-input-5-def8745d09eb>:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_georisques_mensuel = pd.read_csv(path_data+'catastrophes_nat_georisques_mensuel.csv',dtype={"code_insee": str})
<ipython-input-5-def8745d09eb>:3: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_georisques_annuel = pd.read_csv(path_data+'catastrophes_nat_georisques_annuel.csv',dtype={"code_insee": str})


In [6]:
# Charger le fichier permettant d'associer les coordonnées GPS aux codes INSEE
df_incendies = pd.read_csv(path_data+'catastrophes_nat_incendies.csv',dtype={"code_insee": str})


# Regroupement de toutes les catastrophes naturelles

## Fichier annuel

In [7]:
df_catnat_annuel = pd.concat([df_incendies, df_georisques_annuel], ignore_index=True)
df_catnat_annuel

,Code INSEE,Département,Mois,Année,Type catastrophe,Surface
0,40320,40,8.0,2015,Incendie,3000.0
1,40243,40,8.0,2015,Incendie,10000.0
2,40142,40,8.0,2015,Incendie,500.0
3,40310,40,8.0,2015,Incendie,5000.0
4,6115,6,8.0,2015,Incendie,800.0
...,...,...,...,...,...,...
248343,69299,69,NaN,1982,Inondation,NaN
248344,69299,69,NaN,1983,Mouvement de terrain,NaN
248345,69299,69,NaN,1983,Inondation,NaN
248346,69299,69,NaN,1983,Mouvement de terrain,NaN


In [8]:
df_catnat_annuel['Département'].unique()

array(['40', '6', '33', '2B', '4', '13', '34', '2A', '48', '66', '83',
       '11', '30', '19', '46', '5', '47', '17', '38', '49', '73', '84',
       '7', '81', '79', '21', '26', '12', '16', '64', '85', '70', '29',
       '56', '53', '74', '87', '50', '24', '43', '15', '8', '23', '41',
       '71', '63', '65', '18', '32', '37', '01', '02', '03', '04', '05',
       '06', '07', '08', '09', '22', '25', '27', '28', '31', '35', '36',
       '39', '42', '44', '45', '10', '14', '90', '91', 91, 92, 93, 94, 95,
       70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 60, 61, 62, 63,
       64, 65, 66, 67, 68, 69], dtype=object)

In [9]:
# Suppression de la colonne mois
df_catnat_annuel = df_catnat_annuel[["Code INSEE", "Département", "Année", "Type catastrophe", "Surface"]]
df_catnat_annuel

,Code INSEE,Département,Année,Type catastrophe,Surface
0,40320,40,2015,Incendie,3000.0
1,40243,40,2015,Incendie,10000.0
2,40142,40,2015,Incendie,500.0
3,40310,40,2015,Incendie,5000.0
4,6115,6,2015,Incendie,800.0
...,...,...,...,...,...
248343,69299,69,1982,Inondation,NaN
248344,69299,69,1983,Mouvement de terrain,NaN
248345,69299,69,1983,Inondation,NaN
248346,69299,69,1983,Mouvement de terrain,NaN


## Fichier mensuel

In [10]:
df_catnat_mensuel = pd.concat([df_incendies, df_georisques_mensuel], ignore_index=True)
df_catnat_mensuel

,Code INSEE,Département,Mois,Année,Type catastrophe,Surface
0,40320,40,8,2015,Incendie,3000.0
1,40243,40,8,2015,Incendie,10000.0
2,40142,40,8,2015,Incendie,500.0
3,40310,40,8,2015,Incendie,5000.0
4,6115,6,8,2015,Incendie,800.0
...,...,...,...,...,...,...
689632,69299,69,11,1982,Inondation,NaN
689633,69299,69,4,1983,Mouvement de terrain,NaN
689634,69299,69,4,1983,Inondation,NaN
689635,69299,69,5,1983,Mouvement de terrain,NaN


# Ajout du nom de département

In [11]:
departements_dict_inverse = {
    '1': 'Ain', '01': 'Ain',
    '2': 'Aisne', '02': 'Aisne',
    '3': 'Allier', '03': 'Allier',
    '4': 'Alpes-de-Haute-Provence', '04': 'Alpes-de-Haute-Provence',
    '5': 'Hautes-Alpes', '05': 'Hautes-Alpes',
    '6': 'Alpes-Maritimes', '06': 'Alpes-Maritimes',
    '7': 'Ardeche', '07': 'Ardeche',
    '8': 'Ardennes', '08': 'Ardennes',
    '9': 'Ariege', '09': 'Ariege',
    '10': 'Aube',
    '11': 'Aude',
    '12': 'Aveyron',
    '13': 'Bouches-du-Rhone',
    '14': 'Calvados',
    '15': 'Cantal',
    '16': 'Charente',
    '17': 'Charente-Maritime',
    '18': 'Cher',
    '19': 'Correze',
    '2A': 'Corse-du-Sud',
    '21': "Cote-d'Or",
    '22': "Cotes-d'Armor",
    '23': 'Creuse',
    '24': 'Dordogne',
    '25': 'Doubs',
    '26': 'Drome',
    '27': 'Eure',
    '28': 'Eure-et-Loir',
    '29': 'Finistere',
    '30': 'Gard',
    '31': 'Haute-Garonne',
    '32': 'Gers',
    '33': 'Gironde',
    '34': 'Herault',
    '35': 'Ille-et-Vilaine',
    '36': 'Indre',
    '37': 'Indre-et-Loire',
    '38': 'Isere',
    '39': 'Jura',
    '40': 'Landes',
    '41': 'Loir-et-Cher',
    '42': 'Loire',
    '43': 'Haute-Loire',
    '44': 'Loire-Atlantique',
    '45': 'Loiret',
    '46': 'Lot',
    '47': 'Lot-et-Garonne',
    '48': 'Lozere',
    '49': 'Maine-et-Loire',
    '50': 'Manche',
    '51': 'Marne',
    '52': 'Haute-Marne',
    '53': 'Mayenne',
    '54': 'Meurthe-et-Moselle',
    '55': 'Meuse',
    '56': 'Morbihan',
    '57': 'Moselle',
    '58': 'Nievre',
    '59': 'Nord',
    '60': 'Oise',
    '61': 'Orne',
    '62': 'Pas-de-Calais',
    '63': 'Puy-de-Dome',
    '64': 'Pyrenees-Atlantiques',
    '65': 'Hautes-Pyrenees',
    '66': 'Pyrenees-Orientales',
    '67': 'Bas-Rhin',
    '68': 'Haut-Rhin',
    '69': 'Rhône',
    '70': 'Haute-Saone',
    '71': 'Saone-et-Loire',
    '72': 'Sarthe',
    '73': 'Savoie',
    '74': 'Haute-Savoie',
    '75': 'Paris',
    '76': 'Seine-Maritime',
    '77': 'Seine-et-Marne',
    '78': 'Yvelines',
    '79': 'Deux-Sevres',
    '80': 'Somme',
    '81': 'Tarn',
    '82': 'Tarn-et-Garonne',
    '83': 'Var',
    '84': 'Vaucluse',
    '85': 'Vendee',
    '86': 'Vienne',
    '87': 'Haute-Vienne',
    '88': 'Vosges',
    '89': 'Yonne',
    '90': 'Territoire-de-Belfort',
    '91': 'Essonne',
    '92': 'Hauts-de-Seine',
    '93': 'Seine-Saint-Denis',
    '94': 'Val-de-Marne',
    '95': "Val-d'Oise",
    '2B': 'Haute-Corse'
}


In [12]:
# Ajout de la colonne "Nom département"
df_catnat_mensuel["Nom département"] = df_catnat_mensuel["Département"].astype(str).map(departements_dict_inverse)
df_catnat_annuel["Nom département"] = df_catnat_annuel["Département"].astype(str).map(departements_dict_inverse)

In [13]:
df_catnat_mensuel

,Code INSEE,Département,Mois,Année,Type catastrophe,Surface,Nom département
0,40320,40,8,2015,Incendie,3000.0,Landes
1,40243,40,8,2015,Incendie,10000.0,Landes
2,40142,40,8,2015,Incendie,500.0,Landes
3,40310,40,8,2015,Incendie,5000.0,Landes
4,6115,6,8,2015,Incendie,800.0,Alpes-Maritimes
...,...,...,...,...,...,...,...
689632,69299,69,11,1982,Inondation,NaN,Rhône
689633,69299,69,4,1983,Mouvement de terrain,NaN,Rhône
689634,69299,69,4,1983,Inondation,NaN,Rhône
689635,69299,69,5,1983,Mouvement de terrain,NaN,Rhône


# Ajout des coordonnées GPS

# Suppression des Mouvements de terrain

In [14]:
df_catnat_mensuel2 = df_catnat_mensuel[df_catnat_mensuel['Type catastrophe'] != 'Mouvement de terrain']
df_catnat_mensuel2

,Code INSEE,Département,Mois,Année,Type catastrophe,Surface,Nom département
0,40320,40,8,2015,Incendie,3000.0,Landes
1,40243,40,8,2015,Incendie,10000.0,Landes
2,40142,40,8,2015,Incendie,500.0,Landes
3,40310,40,8,2015,Incendie,5000.0,Landes
4,6115,6,8,2015,Incendie,800.0,Alpes-Maritimes
...,...,...,...,...,...,...,...
689630,69299,69,6,1992,Inondation,NaN,Rhône
689631,69299,69,8,1995,Inondation,NaN,Rhône
689632,69299,69,11,1982,Inondation,NaN,Rhône
689634,69299,69,4,1983,Inondation,NaN,Rhône


In [15]:
df_catnat_annuel2 = df_catnat_annuel[df_catnat_annuel['Type catastrophe'] != 'Mouvement de terrain']
df_catnat_annuel2

,Code INSEE,Département,Année,Type catastrophe,Surface,Nom département
0,40320,40,2015,Incendie,3000.0,Landes
1,40243,40,2015,Incendie,10000.0,Landes
2,40142,40,2015,Incendie,500.0,Landes
3,40310,40,2015,Incendie,5000.0,Landes
4,6115,6,2015,Incendie,800.0,Alpes-Maritimes
...,...,...,...,...,...,...
248341,69299,69,1992,Inondation,NaN,Rhône
248342,69299,69,1995,Inondation,NaN,Rhône
248343,69299,69,1982,Inondation,NaN,Rhône
248345,69299,69,1983,Inondation,NaN,Rhône


# Extraction des fichiers

In [16]:
# Exporter le DataFrame en CSV
df_catnat_mensuel2.to_csv(path_export+"catastrophes_naturelles_mensuelles.csv", index=False, encoding='utf-8-sig')
df_catnat_annuel2.to_csv(path_export+"catastrophes_naturelles_annuelles.csv", index=False, encoding='utf-8-sig')

print("Extraction terminée !")

Extraction terminée !
